# AGO Workshop Tools v2

Create 50 Publishers in an ArcGIS Online organization

This script creates bulk users, deltes their content and groups, and deletes those account.  Run the first block of code and then run the required blocks (create, delete content, delete users).

version 2.0 July 2018 (post-UC Revoke fix) /trb   
https://github.com/trbaker/AGOworkshop

In [ ]:
# Run this code block to initialize the script

# import enviros and set vars  ############################################################################## 
from arcgis.gis import GIS
from IPython.display import display
import time

# USER SET VARIABLES ###############################################################
userCount=50                                 # this is the number of anon student accounts to generate. if you want 100, set this number to 100.
orgurl = "http://xxxxxx.maps.arcgis.com"   # use full URL, eg http://mySchoolName.maps.arcgis.com
genericStudentBaseUsername = "GISlearner"     #CHANGE THIS VALUE. This string is the base username. An incremented integer is appended. Example: "TigerStudentAcct" 
genericStudentpw = "GenericPW"            # change this password. It will be the default password for all new student accounts.
genericStudentEmail = "edcommunity@esri.com"  # change this email to a teacher's or administrator's email (eg teacherlastname@myschool.org)
genericStudentLevel = "org_publisher"         # Use only: org_user, org_publisher (no custom roles)
adminuser = ""                                # login for the admin account creating the new users
adminpw = ""                                  # password for the admin account creating new users

# main ##############################################################################       
gis = GIS(orgurl, adminuser, adminpw)

# CREATE USER LIST ##############################################################################
list_users=[]
for n in range(0,userCount):
    list_users.append(genericStudentBaseUsername + str(n))

print(list_users)

In [ ]:
# Run if needed:
# DELETE USER CONTENT ##############################################################################

for j in list_users:
    try:
        print('Deleting content of user: ' + j)
        user = gis.users.get(j)
        for item in user.items():
            try:
                item.protect(enable = False)
                item.delete()
                print(" ")
            except Exception as e:
                print("Item not deleted. It may be delete-protected.")
        # iterate over items in user sub-folders and delete
        for folder in user.folders:
            folderitems = user.items(folder=folder['title'])
            for item in folderitems:
                try:
                    item.delete()
                    print(" ")
                except Exception as e:
                    print("Item not deleted. It may be delete-protected.")
        # delete the folders
        for folder in user.folders:
            try:
                source.content.delete_folder(folder['title'], user)
                print(" ")
            except Exception as e:
                print("A folder was not deleted. It may contain delete-protected content.")
        # delete groups owned
        user_groups = gis.groups.search('owner:' + j, max_groups=50)
        for i in user_groups:
            i.delete()
    except:
        print('An error has been raised.')            


In [ ]:
# run if needed:
# DELETE USERs ##############################################################################

insights_license = gis.admin.license.get('Insights for ArcGIS')
pro_license = gis.admin.license.get('ArcGIS Pro')

for k in list_users:
    try:
        print('Deleting user: ' + k)
        deluser = gis.users.get(k)
        insights_license.revoke(username=k, entitlements='*')
        pro_license.revoke(username=k, entitlements='*')
        deluser.delete()
    except Exception as e:
        print("user delete failed.")
        

In [ ]:
# Run if needed:
# CREATE USERS  ############################################################################## 

def createUser():
    stuName=genericStudentBaseUsername+str(j)
    target_user = gis.users.create(stuName, genericStudentpw, 'ArcGIS', 
        'Student', genericStudentEmail,'Description: temporary user account', genericStudentLevel)
    target_user
    display(target_user)
        
for j in range(0,userCount):
    try:
        createUser()
    except Exception as Ex:
        print(str(Ex))
        print("Unable to create user ")
        
# Add Insights to new accounts ########################################################
insights_license = gis.admin.license.get('Insights for ArcGIS')
pro_license = gis.admin.license.get('ArcGIS Pro')
for i in list_users:
    insights_license.assign(username=i, entitlements='Insights')
    pro_license.assign(username=i, entitlements='desktopAdvN')
    
    